# Create Form 990 Datasets

## Goal:
Ok, we are **SUPER** close to finally getting aggregated Form 990 data of the NPOs we care about!


This file parses Form 990 XML files, and then outputs .csv files of Form 990 data, where each row is an NPO on a specific year, and each column is a variable, such as :
- Fundraising Expenses
- Investment Income
- The number of voting members of the governing body at the end of the tax year.



**A Note that Matters**: Nonprofits file different Form 990s based on various factors. The three primary Form 990s are
- Form 990
- Form 990 EZ
- Form 990 for Private Foundations.

Fortuantely, the code below can process all three types of forms!

**A Cautionary Note:** 

The data in Form 990PFs are drastically different than data in Form 990 and Form 990EZ, so it's probably best for your analysis to drop Form 990PFs if you want to do a overarching analysis, unless you care specifically (or particularly) about private foundations. The fields in the Form 990 and Form 990EZ are quite comparable, and the clean.R code shows you how to compare the two Form datas.


## **Step 1: Setup & Create Dictionary Mapping**

This section initializes paths for data output and mappings for processing.

Below, we see a dictionary mapping. We create various different mappings based on which section of the Form 990 we are in, and whether it is a Form 990, Form 990EZ, or Form 990PF. 


#### Two examples below help elucidate the mappings: 

- **_form_990ez_mapping_** is a dictionary specific to Form 990EZs only. Further, 'Net_Assets_BOY' is the descriptive variable name that will eventually be in the .csv file. The cooresponding XML tag name for Net Assets Beggining of Year is 'NetAssetsOrFundBalancesBOYAmt'.
- **_deep_form_990_mapping_** is a dictionary specific to Form 990s only, not Form 990EZs or Form 990PFs. It is a nested XML structure but is the same conceptually as the other mappings: For example, 'C_S_I_EOY_1' is the descriptive variable name, and the cooresponding XML tag is CashNonInterestBearingGrp, particularly  'EOYAmt'

#### Note on XML Dictionary:
The IRS unfortunately does not give the public XML dictionary for some reason. So, you have to sign up as a developer with the IRS to get the dictionary that  defines what each XML tag name actually means. Sometimes, it's clear what the XML tag name means by its name. For instance, the XML tag **_CYInvestmentIncomeAmt_** is Current Year (CY) Investment Income Amount. For some tags, it is less clear. 

If you want to sign up as a developer undere the IRS to get the XML mapping, visit: https://www.irs.gov/e-file-providers/current-valid-xml-schemas-and-business-rules-for-exempt-organizations-and-other-tax-exempt-entities-modernized-e-file. 

A cruder but much faster approach (than dealing with the IRS) would be to compare a Form 990 XML against a Form 990 pdf form. The steps would include:
- Choose a Form 990 XML file at random
- Go to https://projects.propublica.org/nonprofits/organizations and type in the NPO EIN
- Compare the XML file and PDF file found in Propublica. Check the XML tag and the value within the XML tag against the PDF file to help you define what part of the PDF the XML tag is referring to.

  


In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import chardet


# Important 990s
directory = "\\990_Research\\important_990s\\nonprofits"

# Final dataset path
nonprofit_path = '\\990_Research\\nonprofit_data'

# Output Paths
header_path = nonprofit_path + 'header_data.csv'
filer_path = nonprofit_path + 'filer_data.csv'
form_path = nonprofit_path + 'form_data.csv'
schedule_A_path = nonprofit_path + 'schedule_A_data.csv'
schedule_D_path = nonprofit_path + 'schedule_D_data.csv'
recipient_path = nonprofit_path +'recipient_table_data.csv'
supplemental_path = nonprofit_path + 'supplemental_info_data.csv'
form_ez_path = nonprofit_path + 'form_990ez_data.csv'
form_pf_path = nonprofit_path + 'form_990pf_data.csv'
recipient_pf_path = nonprofit_path + 'recipient_table_pf_data.csv'






# constant for 990, 990EZ
header_mapping = {
    # 'EIN':'EIN', #PRIMARY KEY
    # 'Tax_Year': 'TaxYr', # PRIMARY KEY
    # 'Date_Filed': 'BuildTS', 
    # 'Period_Beg': 'TaxPeriodBeginDt',
    # 'Period_End': 'TaxPeriodEndDt',
    '990_Type': 'ReturnTypeCd'
    # valid options include but are not limited to:
        #990; 990-EZ; 990-T
}

# Constant for 990, 990EZ
filer_mapping = {
    'Name': 'BusinessNameLine1Txt',
    'State': 'StateAbbreviationCd',
    'ZIP_code': 'ZIPCd'
                }

# this mapping is on the IRS990 level, NOT IRS990EZ OR IRS990PF. For those, look at the below other below dictionaries
form_mapping = {
                'Number_Employed': 'TotalEmployeeCnt',
                # Part I line 5: Total number of individuals employed in calendar year 2020
                'Cont_Grants':'CYContributionsGrantsAmt',
                # Part I Line 8: Contributions and grants
                'Prog_Revenue': 'CYProgramServiceRevenueAmt',
                # Part I Line 9: Program Revenue
                'Investment_Income': 'CYInvestmentIncomeAmt',
                # Part I Line 10: Investment Income
                'Other_Revenue': 'CYOtherRevenueAmt',
                #Part I Line 11: Other revenue
               'Total_Revenue': 'CYTotalRevenueAmt',
                # Part I Line 12: Total revenue—add lines 8 through 11 
                'Grants_Expenses': 'CYGrantsAndSimilarPaidAmt',
                # Part I line 13: Grants and similar amounts paid
                'Benefits_Expenses': 'CYBenefitsPaidToMembersAmt',
                # Part I line 14: Benefits paid to or for members
               'Total_Comp': 'CYSalariesCompEmpBnftPaidAmt',
                # Part I Line 15: Salaries, other compensation, employee benefits
               'Other_Expenses_1': 'CYTotalProfFndrsngExpnsAmt',
                # Part I line 16a: Professional fundraising expenses
                'Other_Expenses_2': 'CYTotalFundraisingExpenseAmt',
                # Part I line 16a: Professional fundraising expenses
                'Other_Expenses_3': 'CYOtherExpensesAmt',
                # Part I line 17: Other expenses
               'Total_Expenses': 'CYTotalExpensesAmt',
                # Part I Line 18: Total expenses. Add lines 13–17
               'Net_Revenue': 'CYRevenueLessExpensesAm',
                # Part I Line 19: Revenue less expenses. Subtract line 18 from line 12
                'Total_Assets_EOY': 'TotalAssetsEOYAmt',
                # Part I Line 20: Total assets (Part X, line 16)
                'Net_Assets_BOY': 'NetAssetsOrFundBalancesBOYAmt',
                # Part I Line 22: Net assets or fund balances. Subtract line 21 from line 20
                'Net_Assets_EOY': 'NetAssetsOrFundBalancesBOYAmt',
                # Part I Line 22: Net assets or fund balances. Subtract line 21 from line 20
                'DAF_Dummy': 'DonorAdvisedFundInd',
                # Part IV Line 6: Did the organization maintain any donor advised funds or any similar funds or accounts for which donors have the right to provide advice on the distribution or 
                # Investment of amounts in such funds or accounts? If "Yes," completeSchedule D,Part I
                'Endowment_Dummy': 'DonorRstrOrQuasiEndowmentsInd',
                # Part IV Line 9: Did the organization report an amount in Part X, line 21 for escrow or custodial account liability; serve 
                # as a custodian for amounts not listed in Part X; or provide credit counseling, debt management, credit repair, or debt negotiation services? If "Yes," complete Schedule D, Part IV
                'School_Dummy': 'SchoolOperatingInd',
                # Part IV Linen 13: Did the organization receive more than $25,000 in non-cash contributions? If "Yes," complete Schedule M
                'Noncash_Dummy': 'DeductibleNonCashContriInd',
                # Part IV Line 29: Did the organization receive more than $25,000 in non-cash contributions? If "Yes," complete Schedule M
                'Venture_Dummy': 'InvestmentInJointVentureInd',
                # Part VI Line 16a: Did the organization invest in, contribute assets to, or participate in a joint venture or similar arrangement with a taxable entity during the year? 
                'Other_Gov_Dummy': 'DecisionsSubjectToApprovaInd',
                # Part VI Line 7b: Are any governance decisions of the organization reserved to (or subject to approval by) members, stockholders, or persons other than the governing body?
                'Total_Comp_Alt': 'TotalReportableCompFromOrgAmt',
                # Part VII Line 1d: Total (add lines 1b and 1c)
                'Num_Above_100': 'IndivRcvdGreaterThan100KCnt',
                # Part VII Line 2: Total number of individuals (including but not limited to those listed above) who received more than $100,000 of reportable compensation from the organization.
                'Other_Conts': 'AllOtherContributionsAmt',
                # Part VIII Line 1f: other contributions, gifts, grants, and similar amounts not included above
                'Non_Cash_Conts': 'NoncashContributionsAmt',
                # Part VIII Line 1g: Noncash contributions included in lines 1a - 1f:$
                'Land_Less_Dep': 'LandBldgEquipAccumDeprecAmt',
                # Land, buildings, and equipment: cost or other basis. Complete Part VI of Schedule D   

                # Capturing measures of governance: 
                'Num_Board': 'VotingMembersGoverningBodyCnt',
                # Enter the number of voting members of the governing body at the end of the tax year
                'Num_Indep_Board': 'VotingMembersIndependentCnt'
                # Enter the number of voting members included in line 1a, above, who are independent 
               }


deep_form_990_mapping = {
                    # So I can compare like-with-like for ASSETS, C_S_I_EOY_* is essentially all added up the same as 
                    # the form 990EZ equivalent of Cash, Savings, and Investement Income. Similar logic applies for "other assets"
                   'C_S_I_EOY_1': {'tag_name': 'CashNonInterestBearingGrp', 'subtag_name': 'EOYAmt'},
                    # Part X Line 1: Cash- non-interest bearing
                   'C_S_I_EOY_2': {'tag_name': 'SavingsAndTempCashInvstGrp', 'subtag_name': 'EOYAmt'},
                    # Part X Line 2: Savings and temp cash investments
                   'C_S_I_EOY_3': {'tag_name': 'PledgesAndGrantsReceivableGrp', 'subtag_name': 'EOYAmt'},
                    # Part X Line 3: Net Pledges and Grants Receivable
                   'C_S_I_EOY_4': {'tag_name': 'AccountsReceivableGrp', 'subtag_name': 'EOYAmt'},
                    # Part X Line 4: Accounts Receivable
                   'C_S_I_EOY_5': {'tag_name': 'ReceivablesFromOfficersEtcGrp', 'subtag_name': 'EOYAmt'},
                    # Part X Line 5
                   'C_S_I_EOY_6': {'tag_name': 'RcvblFromDisqualifiedPrsnGrp', 'subtag_name': 'EOYAmt'},
                    # Part X Line 6
                   'C_S_I_EOY_7': {'tag_name': 'OthNotesLoansReceivableNetGrp', 'subtag_name': 'EOYAmt'},
                    # Part X Line 7
                   'C_S_I_EOY_8': {'tag_name': 'InventoriesForSaleOrUseGrp', 'subtag_name': 'EOYAmt'},
                    # Part X Line 8
                   'C_S_I_EOY_9': {'tag_name': 'PrepaidExpensesDefrdChargesGrp', 'subtag_name': 'EOYAmt'},
                    # Part X Line 9
                   'C_S_I_EOY_10': {'tag_name': 'InvestmentsPubTradedSecGrp', 'subtag_name': 'EOYAmt'},
                    # Part X Line 11
                   'C_S_I_EOY_11': {'tag_name': 'InvestmentsOtherSecuritiesGrp', 'subtag_name': 'EOYAmt'},
                    # Part X Line 12
                   'C_S_I_EOY_12': {'tag_name': 'InvestmentsProgramRelatedGrp', 'subtag_name': 'EOYAmt'},
                    # Part X Line 13
                   'Other_Assets_EOY_1': {'tag_name': 'OtherAssetsTotalGrp', 'subtag_name': 'EOYAmt'},
                    # Other Assets
                   'Other_Assets_EOY_2': {'tag_name': 'IntangibleAssetsGrp', 'subtag_name': 'EOYAmt'},
                    # Intangible Assets
                    'L_B_EOY': {'tag_name': 'LandAndBuildingsGrp', 'subtag_name': 'EOYAmt'},
                    # Land less depreciation 


                    # Expenses bc a lot of these help with the governance questions:
                    'O_D_Comp_Exp': {'tag_name': 'CompCurrentOfcrDirectorsGrp', 'subtag_name': 'TotalAmt'},
                    # Part	IX,	line 5,	Column	(A):Compensation of current officers, directors, trustees, and key employees
                    'Travel_Exp': {'tag_name': 'TravelGrp', 'subtag_name': 'TotalAmt'},
                    # Part	IX,	line	17,	Column	(A): Travel Expenses
                    'Admin_Exp': {'tag_name': 'TotalFunctionalExpensesGrp', 'subtag_name': 'ManagementAndGeneralAmt'},
                    # Part	IX,	line	25,	Column	(C): Admininstrative Expenses
                    'Fundraising_Exp': {'tag_name': 'TotalFunctionalExpensesGrp', 'subtag_name': 'FundraisingAmt'},
                    # Part	IX,	line	25,	Column	(D): Fundraising Expenses
                    'Program_Exp': {'tag_name': 'TotalFunctionalExpensesGrp', 'subtag_name': 'ProgramServicesAmt'},
                    # Part	IX,	line	25,	Column	(B): Program Expenses # THIS IS PROGRAM SERVICE EXPENSES btw, which is not the same as program expenses
                    'Inv_Man_Exp':{'tag_name': 'FeesForSrvcInvstMgmntFeesGrp', 'subtag_name': 'TotalAmt'},

        
}



form_990ez_mapping = {
                  'Cont_Grants_1':'ContributionsGiftsGrantsEtcAmt',
                   # Part I Line 1: Contributions and grants
                  'Prog_Revenue': 'ProgramServiceRevenueAmt',
                   # Part I Line 2: Program Revenue
                  'Cont_Grants_2':'MembershipDuesAmt',
                  # Part I Line 3: Membership dues
                  # The reason it is called _2 is because I do _1 + _2 to get the form 990-equivalent of Cont_Grants.
                  'Investment_Income': 'InvestmentIncomeAmt',
                   # Part I Line 4: Investment Income
                   'Other_Revenue_1': 'GainOrLossFromSaleOfAssetsAmt',
                   # Part I Line 5c: Gain or (loss) from sale of assets other than inventory
                   'Other_Revenue_2': 'SpecialEventsNetIncomeLossAmt',
                   # Part I Line 6d: Net income or (loss) from gaming and fundraising events
                   'Other_Revenue_3': 'GrossProfitLossSlsOfInvntryAmt',
                   # Part I Line 7c: Gross profit or (loss) from sales of inventory 
                    'Other_Revenue_4': 'OtherRevenueTotalAmt',
                   # Part I Line 8: Other Revenue
                    'Total_Revenue' :'TotalRevenueAmt',
                   # Part 1 Line 9
                    'Grants_Expenses': 'GrantsAndSimilarAmountsPaidAmt',
                  # Part I line 10: Grants 
                  'Benefits_Expenses': 'BenefitsPaidToOrForMembersAmt',
                  # Part I line 11: Benefits paid to or for members    
                  'Total_Comp': 'SalariesOtherCompEmplBnftAmt',
                   # Part I Line 12: Salaries, other compensation, employee benefits
                   'Other_Expenses_1': 'FeesAndOtherPymtToIndCntrctAmt',
                    # Part I Line 13: Professional Fees
                    'Other_Expenses_2': 'OccupancyRentUtltsAndMaintAmt',
                    # Part I Line 14: Occupancy etc.
                    'Other_Expenses_3': 'PrintingPublicationsPostageAm',
                    # 15
                    'Other_Expenses_4': 'OtherExpensesTotalAmt',
                    # Part I Line 16: Other Expenses
                   'Total_Expenses': 'TotalExpensesAmt',
                   # Part I Line 17: Total expenses.
                  'Net_Assets_BOY': 'NetAssetsOrFundBalancesBOYAmt',
                   # Part I Line 21: Net assets or fund balances
                  'Net_Assets_EOY': 'NetAssetsOrFundBalancesBOYAmt',
                   # Part I Line 21: Net assets or fund balances
                   'Program_Exp': 'TotalProgramServiceExpensesAmt',
                    # Part 3 Line 32
                    'Fundraising_Exp': 'SpecialEventsDirectExpensesAmt',
                    # Fundraising and gaming expenses, Part I Line Less: direct expenses from gaming and fundraising events, close enough to Form 990 Fundraising_Exp variable
                    'O_D_Comp_Exp': 'CYSalariesCompEmpBnftPaidAmt',
                    # Officer Director and employee compensation, close enough to 990 one. Part 1 Line 12.
                    'DAF_Dummy': 'DonorAdvisedFndsInd',
                    'School_Dummy': 'SchoolOperatingInd'
}             

deep_form_990ez_mapping = {
                   'Total_Assets_EOY': {'tag_name': 'Form990TotalAssetsGrp', 'subtag_name': 'EOYAmt'},
                   'C_S_I_EOY': {'tag_name': 'CashSavingsAndInvestmentsGrp', 'subtag_name': 'EOYAmt'},
                    'L_B_EOY': {'tag_name': 'LandAndBuildingsGrp', 'subtag_name': 'EOYAmt'},
                    'Other_Assets_EOY': {'tag_name': 'OtherAssetsTotalDetail', 'subtag_name': 'EOYAmt'}   
}

form_990pf_mapping = {
                  'Cont_Grants':'ContriRcvdRevAndExpnssAmt',
                   # Part I Line 1: Contributions and grants revenue and expenses per book
                  'Total_Rev_Exp': 'TotalRevAndExpnssAmt',
                    # Part I line 12: Total Revenue and expenses per book
                  'Total_Comp': 'CompOfcrDirTrstRevAndExpnssAmt',
                  # Part I line 13: Compensation of officers, directors, trustees, etc. revenue and expenses per book
                  'Excess_Rev_Expenses':'ExcessRevenueOverExpensesAmt',
                  # Part I Line 27: Excess of revenue over expenses and disbursements

}   


# # deep_form_mapping = {<InvestmentIncomeGrp><TotalRevenueColumnAmt>221763</TotalRevenueColumnAmt>
# #     # Part VIII Line 3: Investment income (including dividends, interest, and othersimilar amounts)
# <LandBldgEquipBasisNetGrp><EOYAmt>8240266</EOYAmt>
# #Part X Line 10c:Land, buildings, and equipment: cost or other basis. Complete Part VI of Schedule 
# <InvestmentsPubTradedSecGrp><EOYAmt>7478</EOYAmt>
# # Part X Line 11: Investments- Publicly traded securities
# <InvestmentsOtherSecuritiesGrp><EOYAmt>30450321</EOYAmt>,
# <OtherAssetsTotalGrp><EOYAmt>1805878</EOYAmt>
# Part X Line 15: Other Assets
# # }


# Public Charity Status and Public Support. Complete if the organization is a section 501(c)(3) organization or a section 4947(a)(1) nonexempt charitable trust.
schedule_A_mapping = {
                      'Perc_Public_Support': 'PublicSupportCY170Pct',
                      # % Percent public support

    
# <TotalSupportCalendarYearGrp>
# <CurrentTaxYearMinus4YearsAmt>162436</CurrentTaxYearMinus4YearsAmt>
# <CurrentTaxYearMinus3YearsAmt>147206</CurrentTaxYearMinus3YearsAmt>
# <CurrentTaxYearMinus2YearsAmt>140395</CurrentTaxYearMinus2YearsAmt>
# <CurrentTaxYearMinus1YearAmt>134254</CurrentTaxYearMinus1YearAmt>
# <CurrentTaxYearAmt>133419</CurrentTaxYearAmt>
# <TotalAmt>717710</TotalAmt>
# The above are amount of "Total Support", not Total Reevenue. Total support is gross income from XXX, net income from unr business, other income. Not sure how different it is from total revenue, but probably similar. 
}

# Supplemental Financial Statements SchDMd Bullet Complete if the organization answered "Yes," on Form 990, Part IV, line 6, 7, 8, 9, 10, 11a, 11b, 11c, 11d, 11e, 11f, 12a, or 12b.
schedule_D_mapping = {
                      'Perc_Board_Quasi_End': 'BoardDesignatedBalanceEOYPct',
                       # Part V Line 2a: Provide the estimated percentage of the current year end balance (line 1g, column (a)) held as: a Board designated or quasi-endowment
                      'Perc_Perm_End': 'PrmnntEndowmentBalanceEOYPct',
                       # Part V Line 2a: Provide the estimated percentage of the current year end balance (line 1g, column (a)) held as: a Permanennt Endowment
                       # note that you can create Perc_Term_End = 100 - Quasi - Perm
                       'Unrelated_Manage_End': 'EndowmentsHeldUnrelatedOrgInd',
                       # Are there endowment funds not in the possession of the organization that are held and administered for the organization by (i) Unrelated organizations 
                       'Related_Manage_End': 'EndowmentsHeldRelatedOrgInd'
                       # Are there endowment funds not in the possession of the organization that are held and administered for the organization by (ii) Unrelated organizations 
                        
                    
}


deep_schedule_D_mapping = {
    'M0_End_Value_BOY': {'tag_name': 'CYEndwmtFundGrp', 'subtag_name': 'BeginningYearBalanceAmt'},
    # Beginning of year balance current year
    'M0_End_Value_EOY': {'tag_name': 'CYEndwmtFundGrp', 'subtag_name': 'EndYearBalanceAmt'},
    # End of year balance current year
    'M0_Earnings': {'tag_name': 'CYEndwmtFundGrp', 'subtag_name': 'InvestmentEarningsOrLossesAmt'},
    # Net investment earnings, gains, and losses
    'M0_Contributions': {'tag_name': 'CYEndwmtFundGrp', 'subtag_name': 'ContributionsAmt'},
    
    'M1_End_Value_BOY': {'tag_name': 'CYMinus1YrEndwmtFundGrp', 'subtag_name': 'BeginningYearBalanceAmt'},
    # Beginning of year balance previous year
    'M1_End_Value_EOY': {'tag_name': 'CYMinus1YrEndwmtFundGrp', 'subtag_name': 'EndYearBalanceAmt'},
    # End of year balance current year
    'M1_Earnings': {'tag_name': 'CYMinus1YrEndwmtFundGrp', 'subtag_name': 'InvestmentEarningsOrLossesAmt'},
    # Net investment earnings, gains, and losses
    'M1_Contributions': {'tag_name': 'CYMinus1YrEndwmtFundGrp', 'subtag_name': 'ContributionsAmt'},
    # Contributions

    'M2_End_Value_BOY': {'tag_name': 'CYMinus2YrEndwmtFundGrp', 'subtag_name': 'BeginningYearBalanceAmt'},
    # Beginning of year balance two years prior
    'M2_End_Value_EOY': {'tag_name': 'CYMinus2YrEndwmtFundGrp', 'subtag_name': 'EndYearBalanceAmt'},
    # Beginning of year balance two years prior
    'M2_Earnings': {'tag_name': 'CYMinus2YrEndwmtFundGrp', 'subtag_name': 'InvestmentEarningsOrLossesAmt'},
    # Net investment earnings, gains, and losses
    'M2_Contributions': {'tag_name': 'CYMinus2YrEndwmtFundGrp', 'subtag_name': 'ContributionsAmt'},
     # Contributions   
    
    'M3_End_Value_BOY': {'tag_name': 'CYMinus3YrEndwmtFundGrp', 'subtag_name': 'BeginningYearBalanceAmt'},
    # Beginning of year balance three years prior
    'M3_End_Value_EOY': {'tag_name': 'CYMinus3YrEndwmtFundGrp', 'subtag_name': 'EndYearBalanceAmt'},
    # Beginning of year balance two years prior
    'M3_Earnings': {'tag_name': 'CYMinus3YrEndwmtFundGrp', 'subtag_name': 'InvestmentEarningsOrLossesAmt'},
    # Net investment earnings, gains, and losses
    'M3_Contributions': {'tag_name': 'CYMinus3YrEndwmtFundGrp', 'subtag_name': 'ContributionsAmt'},
    # Contributions
    
    'M4_End_Value_BOY': {'tag_name': 'CYMinus4YrEndwmtFundGrp', 'subtag_name': 'BeginningYearBalanceAmt'},
    # Beginning of year balance four years prior
    'M4_End_Value_EOY': {'tag_name': 'CYMinus4YrEndwmtFundGrp', 'subtag_name': 'EndYearBalanceAmt'},
    # Beginning of year balance two years prior
    'M4_Earnings': {'tag_name': 'CYMinus4YrEndwmtFundGrp', 'subtag_name': 'InvestmentEarningsOrLossesAmt'},
    # Net investment earnings, gains, and losses
    'M4_Contributions': {'tag_name': 'CYMinus4YrEndwmtFundGrp', 'subtag_name': 'ContributionsAmt'}
    # Contributions
    
    # can include expenses related to endowments, but don't worry about it for now.                
}



# #Schedule M: Noncash Contributions Right pointing arrow large imageComplete if the organizations answered "Yes" on Form 990, Part IV, lines 29 or 30.
# # Extremeley Granular, and we can parse by noncash contribution (they have car, art, real estate, boats, clothes, etc.)
# deep_schedule_M_mapping = {<SecuritiesPubliclyTradedGrp><ContributionCnt>24</ContributionCnt><NoncashContributionsRptF990Amt>473318</NoncashContributionsRptF990Amt>
# }
# <ArtHistoricalTreasuresGrp/>
# <ArtFractionalInterestGrp/>
# <BooksAndPublicationsGrp/>
# <ClothingAndHouseholdGoodsGrp/>
# <CarsAndOtherVehiclesGrp/>
# <BoatsAndPlanesGrp/>
# <IntellectualPropertyGrp/>
# <SecuritiesCloselyHeldStockGrp/>
# <SecurPrtnrshpTrustIntrstsGrp/>
# <SecuritiesMiscellaneousGrp/>
# <QualifiedContribHistStructGrp/>
# <QualifiedContribOtherGrp/>
# <RealEstateResidentialGrp/>
# <RealEstateCommercialGrp/>
# <RealEstateOtherGrp/>
# <CollectiblesGrp/>
# <FoodInventoryGrp/>
# <DrugsAndMedicalSuppliesGrp/>
# <TaxidermyGrp/>
# <HistoricalArtifactsGrp/>
# <ScientificSpecimensGrp/>
# <ArchaeologicalArtifactsGrp/>



recipient_table_mapping = {
                      'Name': 'RecipientBusinessName',
                      # This variable is scattered throughout the Form 990s and its schedules and is described as:  Provide the descriptions required for Schedule X Part Y, Line Z
                      'State': 'StateAbbreviationCd',
                      # This is where the filer explains more in depth whatever they need to explain.
                      'Recipient_EIN': 'RecipientEIN',
                       'IRS_Classification': 'IRCSectionDesc',
                      'Cash_Received': 'CashGrantAmt'
}



recipient_tablepf_mapping = {
                             'Name' :'BusinessNameLine1Txt',
                              # Part XIV: Name of Business
                             'Address': 'AddressLine1Txt',
                              # Part XIV: Address
                              'Description': 'GrantOrContributionPurposeTxt',
                                # Purpose of grant or contribution
                              'Cash_Received': 'Amt'
                                # Amount

                                ## Note that I do not map grants and contributions to individuals, only grants/cont to entities
}



# Supplemental Info. For now, the only supplemental information we care about is for Part V, Line 4, i.e. "Describe in Part XIII the intended uses of the organization's endowment funds
supplemental_info_mapping = {
                      'Form_Line': 'FormAndLineReferenceDesc',
                      # This variable is scattered throughout the Form 990s and its schedules and is described as:  Provide the descriptions required for Schedule X Part Y, Line Z
                      'Explanation': 'ExplanationTxt'
                      # This is where the filer explains more in depth whatever they need to explain.
}
    


optional_filer_mapping = ['State','ZIP_code']



header_data = []
filer_data = []
form_data = []

schedule_A_data = []
schedule_D_data = []
schedule_M_data = []

recipient_table_data = []
recipient_table_pf_data = []
supplemental_info_data = []


form_990ez_data = []
form_990pf_data = []






## Step 2: Iterate Through Files

Now comes the real fun: we simply loops through all XML files in the input directory and parse their contents. This is a pretty customary loop for parsing XML files. 

The loop does a couple things:
- Handles data extraction for specific forms and schedules, using mappings to populate dictionaries.
- Handles optional tags using try-except to avoid breaking the loop.
- Seperates the logic for all three form types



In [ ]:


# encoding matters for later parsing
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        raw_data = f.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
        return encoding



for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    encoding = detect_encoding(file_path)
    i = i + 1

    
    with open(file_path, 'r', encoding=encoding) as file: 
        
        contents = file.read()
        print(filename)
        soup = BeautifulSoup(contents,'xml')

        # Find the various sections of the Form 990
        header = soup.find('ReturnHeader')
        filer = soup.find('Filer')
        form = soup.find('IRS990')
        form_990ez = soup.find('IRS990EZ')
        form_990pf = soup.find('AnalysisOfRevenueAndExpenses')

        # Find Schedules
        schedule_A = soup.find('IRS990ScheduleA')
        schedule_D = soup.find('IRS990ScheduleD')
        schedule_M = soup.find('IRS990ScheduleM')
        schedule_I = soup.find('IRS990ScheduleI')
        
        if schedule_I is not None:
            # Parse recipient table within Schedule I
            recipient_table = soup.find_all('RecipientTable')
        else:
            recipient_table = None

        # Find Supplemental Info
        supplemental_info = soup.find_all('SupplementalInformationDetail')

        # Find recipient table for Form 990PF
        recipient_table_pf = soup.find_all('GrantOrContributionPdDurYrGrp')
        
        # Primary keys need to be distributed to each dataset
        EIN = header.find('EIN').text
        tax_year = soup.find('TaxYr').text
        

        # Name (not a primary key) should also be distributed to each dataset
        Name = filer.find('BusinessNameLine1Txt').text

        
        # Parse header (990, 990EZ, and 990PF have same headers)    
        header_row = {}
        for variable, tag in header_mapping.items():
            header_row[variable] = header.find(tag).text
        header_data.append(header_row)
        
       
        # Parse filer information (990, 990EZ, and 990PF have same filer info) 
        filer_row = {}
        filer_row["EIN"] = EIN
        filer_row["Tax_Year"] = tax_year
        for variable, tag in filer_mapping.items():
            # State & Zip Code are sometimes missing, so assign them to optional tagging
            if variable in optional_filer_mapping:
                try: 
                    filer_row[variable] = filer.find(tag).text
                except(AttributeError, TypeError):
                    filer_row[variable] = None
            else:
                filer_row[variable] = filer.find(tag).text
        filer_data.append(filer_row)


        
        #**parsing is different based on form type**
        Form_Type = header.find('ReturnTypeCd').text
        print(Form_Type)


        # # Form 990
        if Form_Type == "990":


            
        #     # Parse IRS 990 
            form_row = {}
            form_row["EIN"] = EIN
            form_row["Name"] = Name
            form_row["Tax_Year"] = tax_year
            form_row["990_Type"] = Form_Type
            for variable, tag in form_mapping.items():
                # I am assuming all tags are optional. If tag not found, this implies they don't have info.
                try: 
                    form_row[variable] = form.find(tag).text
                except(AttributeError, TypeError):
                    form_row[variable] = None

            
            # # Parse deep tags of 990
            for variable, mapping_info in deep_form_990_mapping.items():
                try:
                    tag = form.find(mapping_info['tag_name'])
                    subtag = tag.find(mapping_info['subtag_name']).text
                    form_row[variable] = subtag
                except(AttributeError, TypeError):
                    form_row[variable] = None
            form_data.append(form_row)
        
            # Parse Schedule A
            schedule_A_row = {}
            schedule_A_row["EIN"] = EIN
            schedule_A_row["Name"] = Name
            schedule_A_row["Tax_Year"] = tax_year
            schedule_A_row["990_Type"] = Form_Type
            for variable, tag in schedule_A_mapping.items():
                # I am assuming all tags are optional. If tag not found, this implies they don't have info.
                try: 
                    schedule_A_row[variable] = schedule_A.find(tag).text
                except(AttributeError, TypeError):
                    schedule_A_row[variable] = None
            schedule_A_data.append(schedule_A_row)
            

            # # Parse Schedule D
            schedule_D_row = {}
            schedule_D_row["EIN"] = EIN
            schedule_D_row["Name"] = Name
            schedule_D_row["Tax_Year"] = tax_year
            schedule_D_row["990_Type"] = Form_Type
            for variable, tag in schedule_D_mapping.items():
                # I am assuming all tags are optional. If tag not found, this implies they don't have info.
                try: 
                    schedule_D_row[variable] = schedule_D.find(tag).text
                except(AttributeError, TypeError):
                    schedule_D_row[variable] = None

            # # Parse deep tags of Schedule D
            for variable, mapping_info in deep_schedule_D_mapping.items():
                try:
                    tag = schedule_D.find(mapping_info['tag_name'])
                    subtag = tag.find(mapping_info['subtag_name']).text
                    schedule_D_row[variable] = subtag
                except(AttributeError, TypeError):
                    schedule_D_row[variable] = None
            schedule_D_data.append(schedule_D_row)


            # # Parse Recipients of Grants (within Schedule I)
            if recipient_table is not None:
                
                for grantee in recipient_table:
                    recipient_table_row = {}
                    recipient_table_row["EIN"] = EIN
                    recipient_table_row["Name"] = Name
                    recipient_table_row["Tax_Year"] = tax_year
                    recipient_table_row["990_Type"] = Form_Type
                    for variable, tag in recipient_table_mapping.items():
                        # I am assuming all tags are optional. If tag not found, this implies they don't have info.
                        try: 
                            recipient_table_row[variable] = grantee.find(tag).text
                        except(AttributeError, TypeError):
                            recipient_table_row[variable] = None
                    recipient_table_data.append(recipient_table_row)
        
        #     # Parse Supplemental Information
            for sup_info in supplemental_info:


                
                supplemental_info_row = {}
                supplemental_info_row["EIN"] = EIN
                supplemental_info_row["Name"] = Name
                supplemental_info_row["Tax_Year"] = tax_year
                supplemental_info_row["990_Type"] = Form_Type
                for variable, tag in supplemental_info_mapping.items():
                    # I am assuming all tags are optional. If tag not found, this implies they don't have info.
                    try: 
                        supplemental_info_row[variable] = sup_info.find(tag).text
                    except(AttributeError, TypeError):
                        supplemental_info_row[variable] = None
                supplemental_info_data.append(supplemental_info_row)
                
        if Form_Type == "990EZ":
            
        #     # Parse IRS 990EZ 
            form_990ez_row = {}
            form_990ez_row["EIN"] = EIN
            form_990ez_row["Name"] = Name
            form_990ez_row["Tax_Year"] = tax_year
            form_990ez_row["990_Type"] = Form_Type 
            for variable, tag in form_990ez_mapping.items():
                # I am assuming all tags are optional. If tag not found, this implies they don't have info.
                try: 
                    form_990ez_row[variable] = form_990ez.find(tag).text
                except(AttributeError, TypeError):
                    form_990ez_row[variable] = None

            # # Parse deep tags of 990EZ
            for variable, mapping_info in deep_form_990ez_mapping.items():
                try:
                    tag = form_990ez.find(mapping_info['tag_name'])
                    subtag = tag.find(mapping_info['subtag_name']).text
                    form_990ez_row[variable] = subtag
                except(AttributeError, TypeError):
                    form_990ez_row[variable] = None
            form_990ez_data.append(form_990ez_row)

        #     # Parse Schedule A
            schedule_A_row = {}
            schedule_A_row["EIN"] = EIN
            schedule_A_row["Name"] = Name
            schedule_A_row["Tax_Year"] = tax_year
            schedule_A_row["990_Type"] = Form_Type
            for variable, tag in schedule_A_mapping.items():
#                 # I am assuming all tags are optional. If tag not found, this implies they don't have info.
                try: 
                    schedule_A_row[variable] = schedule_A.find(tag).text
                except(AttributeError, TypeError):
                    schedule_A_row[variable] = None
            schedule_A_data.append(schedule_A_row)


            # # Parse Schedule D
            schedule_D_row = {}
            schedule_D_row["EIN"] = EIN
            schedule_D_row["Name"] = Name
            schedule_D_row["Tax_Year"] = tax_year
            schedule_D_row["990_Type"] = Form_Type
            for variable, tag in schedule_D_mapping.items():
                # I am assuming all tags are optional. If tag not found, this implies they don't have info.
                try: 
                    schedule_D_row[variable] = schedule_D.find(tag).text
                except(AttributeError, TypeError):
                    schedule_D_row[variable] = None

            # # Parse deep tags of Schedule D
            for variable, mapping_info in deep_schedule_D_mapping.items():
                try:
                    tag = schedule_D.find(mapping_info['tag_name'])
                    subtag = tag.find(mapping_info['subtag_name']).text
                    schedule_D_row[variable] = subtag
                except(AttributeError, TypeError):
                    schedule_D_row[variable] = None
            schedule_D_data.append(schedule_D_row)

    
        if Form_Type == "990PF":

#             # Parse IRS 990PF
            form_990pf_row = {}
            form_990pf_row["EIN"] = EIN
            form_990pf_row["Name"] = Name
            form_990pf_row["Tax_Year"] = tax_year
            form_990pf_row["990_Type"] = Form_Type 
            for variable, tag in form_990pf_mapping.items():
#                 # I am assuming all tags are optional. If tag not found, this implies they don't have info.
                try: 
                    form_990pf_row[variable] = form_990pf.find(tag).text
                except(AttributeError, TypeError):
                    form_990pf_row[variable] = None
            form_990pf_data.append(form_990pf_row)

            for grantee in recipient_table_pf:
                recipient_table_pf_row = {}
                recipient_table_pf_row["EIN"] = EIN
                recipient_table_pf_row["Name"] = Name
                recipient_table_pf_row["Tax_Year"] = tax_year
                recipient_table_pf_row["990_Type"] = Form_Type
                for variable, tag in recipient_tablepf_mapping.items():
                    # I am assuming all tags are optional. If tag not found, this implies they don't have info.
                    try: 
                        recipient_table_pf_row[variable] = grantee.find(tag).text
                    except(AttributeError, TypeError):
                        recipient_table_pf_row[variable] = None
                recipient_table_pf_data.append(recipient_table_pf_row)




## **Step 3**: Save Datasets


Convert the data in pandas dataframes and export to .csv.


In [ ]:

#-------------All Form Data----------#
df0 = pd.DataFrame(header_data)
headers_data = df0.to_csv(header_path)

df1 = pd.DataFrame(filer_data)
filers_data = df1.to_csv(filer_path)

#-------------Form 990 Data----------#
df2 = pd.DataFrame(form_data)
forms_data = df2.to_csv(form_path)

#-------------Form 990 Schedule Data----------#
dfA = pd.DataFrame(schedule_A_data)
schedule_As_data = dfA.to_csv(schedule_A_path)

dfD = pd.DataFrame(schedule_D_data)
schedule_Ds_data = dfD.to_csv(schedule_D_path)

#-------------Form 990 Recipient Table & Supplemental Data----------#
dfR = pd.DataFrame(recipient_table_data)
recipient_tables_data = dfR.to_csv(recipient_path)

dfS = pd.DataFrame(supplemental_info_data)
supplemental_infos_data = dfS.to_csv(supplemental_path)


#-------------Form 990EZ Data----------#
df3 = pd.DataFrame(form_990ez_data)
form_990ezs_data = df3.to_csv(form_ez_path)


#-------------Form 990PF Data----------#
df4 = pd.DataFrame(form_990pf_data)
form_990pf_data = df4.to_csv(form_pf_path)

#-------------Form 990PF Recipient Data----------#
dfRPF = pd.DataFrame(recipient_table_pf_data)
recipient_tablespf_data = dfRPF.to_csv(recipient_pf_path)
